### ssh eddies ###

__data__

In [1]:
import datetime, jdcal
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
ds = xr.open_dataset('/home/cz2397/data/ssh-eddies/tracks_AVISO_DT2014_daily_web.nc', decode_cf=False)

In [3]:
df = ds.to_dataframe()

In [4]:
df.head()

,track,n,j1,cyc,lon,lat,A,L,U
Nobs,,,,,,,,,
0,1,1,2448989,1,397.040985,-21.719400,31.473101,146.738998,79.148399
1,1,2,2448990,1,397.015991,-21.760300,30.215599,138.673996,79.141296
2,1,3,2448991,1,397.003998,-21.797701,28.817200,137.839996,78.532097
3,1,4,2448992,1,396.971008,-21.852100,28.791300,136.145996,77.241402
4,1,5,2448993,1,396.945007,-21.871201,27.741199,136.119995,77.705101


__time__

In [5]:
def jday_to_datetime(jday, refday=0):
    y, m, d, f = jdcal.jd2gcal(jday, refday)
    h = int(f*24)
    return pd.to_datetime(datetime.datetime(y, m, d, h))

In [6]:
date_fix = df.j1.apply(jday_to_datetime)

In [7]:
df.j1 = date_fix
df = df.rename(columns = {'j1': 'date'})

In [8]:
df.head()

,track,n,date,cyc,lon,lat,A,L,U
Nobs,,,,,,,,,
0,1,1,1993-01-01 12:00:00,1,397.040985,-21.719400,31.473101,146.738998,79.148399
1,1,2,1993-01-02 12:00:00,1,397.015991,-21.760300,30.215599,138.673996,79.141296
2,1,3,1993-01-03 12:00:00,1,397.003998,-21.797701,28.817200,137.839996,78.532097
3,1,4,1993-01-04 12:00:00,1,396.971008,-21.852100,28.791300,136.145996,77.241402
4,1,5,1993-01-05 12:00:00,1,396.945007,-21.871201,27.741199,136.119995,77.705101


__longitude__

In [9]:
# longitude range setting
# from -180 to 180
lon_fix_1 = df.where(df.lon < 540).lon - 360
lon_fix_2 = df.where(df.lon >= 540).lon - 720
lon_fix = lon_fix_1
lon_fix = lon_fix_1.fillna(lon_fix_2)
df.lon = lon_fix

In [10]:
df.head()

,track,n,date,cyc,lon,lat,A,L,U
Nobs,,,,,,,,,
0,1,1,1993-01-01 12:00:00,1,37.040985,-21.719400,31.473101,146.738998,79.148399
1,1,2,1993-01-02 12:00:00,1,37.015991,-21.760300,30.215599,138.673996,79.141296
2,1,3,1993-01-03 12:00:00,1,37.003998,-21.797701,28.817200,137.839996,78.532097
3,1,4,1993-01-04 12:00:00,1,36.971008,-21.852100,28.791300,136.145996,77.241402
4,1,5,1993-01-05 12:00:00,1,36.945007,-21.871201,27.741199,136.119995,77.705101


__radius__

In [11]:
radius = df.L*1000
df.L = radius
df = df.rename(columns = {'L': 'radius'})

In [12]:
df.head()

,track,n,date,cyc,lon,lat,A,radius,U
Nobs,,,,,,,,,
0,1,1,1993-01-01 12:00:00,1,37.040985,-21.719400,31.473101,146739,79.148399
1,1,2,1993-01-02 12:00:00,1,37.015991,-21.760300,30.215599,138674,79.141296
2,1,3,1993-01-03 12:00:00,1,37.003998,-21.797701,28.817200,137840,78.532097
3,1,4,1993-01-04 12:00:00,1,36.971008,-21.852100,28.791300,136146,77.241402
4,1,5,1993-01-05 12:00:00,1,36.945007,-21.871201,27.741199,136120,77.705101


### mongodb ###

In [13]:
from pymongo import MongoClient
from tqdm import tqdm

In [14]:
track = np.asarray(df.track)
n = np.asarray(df.n)
date = df.date
lon = np.asarray(df.lon)
lat = np.asarray(df.lat)
radius = np.asarray(df.radius)

In [15]:
tra_num = max(track)
tra_num

245389

In [16]:
obs_num = len(n)
obs_num

22510604

In [17]:
df.head()

,track,n,date,cyc,lon,lat,A,radius,U
Nobs,,,,,,,,,
0,1,1,1993-01-01 12:00:00,1,37.040985,-21.719400,31.473101,146739,79.148399
1,1,2,1993-01-02 12:00:00,1,37.015991,-21.760300,30.215599,138674,79.141296
2,1,3,1993-01-03 12:00:00,1,37.003998,-21.797701,28.817200,137840,78.532097
3,1,4,1993-01-04 12:00:00,1,36.971008,-21.852100,28.791300,136146,77.241402
4,1,5,1993-01-05 12:00:00,1,36.945007,-21.871201,27.741199,136120,77.705101


In [18]:
# collection
ssh_cchdo = MongoClient().eddies.ssh_cchdo

In [19]:
for i in tqdm(range(obs_num)):
    eddy = {
        'eid': str(track[i]) + "-" + str(n[i]),
        'dat': date[i],
        'loc': {
            'type': 'Point',
            'coordinates': [float(lon[i]), float(lat[i])]
        },
        'rad': float(radius[i])
    }
    ssh_cchdo.insert_one(dict(eddy))

100%|██████████| 22510604/22510604 [1:41:05<00:00, 3711.45it/s]
